# Project: Multimodal Counterfactual Sentiment Analysis

## Installation

In [ ]:
!pip install nltk transformers torch shap matplotlib pandas scikit-learn
!pip install librosa numpy torch transformers
!pip install av                                 # PyAV (FFmpeg bindings)
!pip install transformers                       # Hugging Face Transformers
!pip install pytorchvideo                       # PyTorchVideo (video processor)
!pip install opencv-python-headless             # OpenCV I/O
!pip install shap                               # SHAP for explainability
!pip install torch                              # PyTorch backend
!pip install matplotlib                         # Plotting

## Imports

In [ ]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from nltk.corpus import wordnet as wn
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
import shap
from sklearn.metrics import accuracy_score, f1_score
from transformers import pipeline
from collections import Counter
import librosa
import torch.nn as nn
from transformers import Wav2Vec2FeatureExtractor, HubertModel
import cv2
from transformers import CLIPProcessor, CLIPModel

## Section: Text Counterfactual Analysis

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')  # for antonyms



In [ ]:
# ===== Part 2: Model & Prediction Function =====
MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer  = AutoTokenizer.from_pretrained(MODEL_NAME)
model      = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
device     = 0 if torch.cuda.is_available() else -1

pipe = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    return_all_scores=True,
    device=device
)

LABEL_ORDER = ["LABEL_0", "LABEL_1", "LABEL_2"]
LABELS      = ["negative", "neutral", "positive"]

def predict_proba(texts):
    """
    Given a list of texts, returns an (n_texts x 3) array of [neg, neu, pos] probabilities.
    """
    raw = pipe(texts)
    proba = []
    for o in raw:
        score_map = {d['label']: d['score'] for d in o}
        proba.append([score_map[lbl] for lbl in LABEL_ORDER])
    return np.array(proba)

In [ ]:
# ===== Part 3: Original Prediction & SHAP Bar Chart =====
# Target sentence
text = (
    "I love you"
)

# 3.1 Single-example prediction
orig_probs = predict_proba([text])[0]
orig_lbl   = LABELS[np.argmax(orig_probs)]
print(f"Original prediction: {orig_lbl.upper()}   {dict(zip(LABELS, orig_probs.round(3)))}")

# 3.2 Probability bar chart
plt.figure(figsize=(6,4))
plt.bar(LABELS, orig_probs)
plt.ylim(0,1)
plt.title("Original sentiment probabilities")
plt.show()

# 3.3 SHAP values bar chart for tokens
shap.initjs()
explainer = shap.Explainer(pipe, output_names=LABELS)
sv        = explainer([text])[0]         # single Explanation
pred_idx  = np.argmax(orig_probs)
# Extract shapley values for each token for the predicted class
shap_vals = sv.values[:, pred_idx]       # shape = (n_tokens,)
tokens    = list(sv.data)
abs_vals  = np.abs(shap_vals)

# Bar chart of absolute SHAP values per token\ nplt.figure(figsize=(10,4))
plt.bar(range(len(tokens)), abs_vals)
plt.xticks(range(len(tokens)), tokens, rotation=45, ha='right')
plt.ylabel('abs(SHAP value)')
plt.title('Token importance (|SHAP|) for predicted class')
plt.tight_layout()
plt.show()

In [ ]:
# ===== Part 4: SHAP Force Plot =====
# Local force-plot showing positive/negative contributions
# ===== Add JavaScript Initialization =====

# Initialize JS visualization code (MUST COME FIRST)
shap.initjs()

# ===== Then run your visualization code =====
shap.force_plot(
    base_value=base_value,
    shap_values=shap_values,
    features=shap_explanation.data,
    feature_names=shap_explanation.feature_names
)

In [ ]:
# ===== Revised Cell 5: Counterfactual via Masked-LM Antonym =====


# 5.1: Extract SHAP explanation & tokens for the predicted class
sv_cf = explainer([text])[0]                # SHAP Explanation object
shap_arr = sv_cf.values[:, pred_idx]         # SHAP values for pred class
tokens_cf = list(sv_cf.data)                 # token list
abs_arr = np.abs(shap_arr)

# 5.2: Identify the top-impact token (skip subwords starting with ##)
max_score = -np.inf
idx_max = -1
for i, (score, token) in enumerate(zip(abs_arr, tokens_cf)):
    # Skip subword tokens (marked with ## in BERT-style tokenization)
    if not token.startswith("##") and score > max_score:
        max_score = score
        idx_max = i
# Fallback if all tokens are subwords
if idx_max == -1:
    idx_max = np.argmax(abs_arr)
token_to_change = tokens_cf[idx_max]
print(f"⚠ Top-impact token: '{token_to_change}'")

# 5.3: Improved masked-LM antonym generation
mask_pipe = pipeline(
    "fill-mask",
    model="bert-base-uncased",
    device=-1  # CPU
)

def get_antonym_ml(word):
    """Improved antonym generation with better prompt and validation"""
    # Try multiple prompt templates
    templates = [
        f"The antonym of {word} is [MASK].",
        f"{word} is the opposite of [MASK].",
        f"Opposite word: {word} → [MASK]."
    ]

    candidates = []
    for prompt in templates:
        results = mask_pipe(prompt, top_k=5)
        for res in results:
            candidate = res["token_str"].strip().lower()
            # Basic validation: antonym shouldn't be same as original
            if candidate != word.lower():
                candidates.append(candidate)

    # Return most common valid candidate
    if candidates:
        return max(set(candidates), key=candidates.count)
    else:
        return word  # fallback

antonym = get_antonym_ml(token_to_change)
print(f"💡 Replacing '{token_to_change}' → '{antonym}'")

# 5.4: Case-insensitive replacement with lowercase antonym
cf_tokens = [
    antonym if tok.lower() == token_to_change.lower() else tok
    for tok in tokens_cf
]
cf_text = " ".join(cf_tokens).replace(" ##", "")  # Clean subword markers

# 5.5: Score original vs. counterfactual
orig_probs = predict_proba([text])[0]
cf_probs = predict_proba([cf_text])[0]

print("Original probs:      ", dict(zip(LABELS, orig_probs.round(3))))
print("Counterfactual probs:", dict(zip(LABELS, cf_probs.round(3))))

# 5.6: Plot comparison
x = np.arange(len(LABELS))
width = 0.35

plt.figure(figsize=(6,3))
plt.bar(x - width/2, orig_probs, width, label="Original")
plt.bar(x + width/2, cf_probs, width, label="Counterfactual")
plt.xticks(x, LABELS)
plt.ylim(0,1)
plt.title(f"Effect of replacing '{token_to_change}' with '{antonym}'")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# ===== Improved Counterfactual with Antonym Focus =====


# 5.1: Extract SHAP explanation & tokens
sv_cf = explainer([text])[0]
shap_arr = sv_cf.values[:, pred_idx]
tokens_cf = list(sv_cf.data)
abs_arr = np.abs(shap_arr)

# 5.2: Find top-impact token (skip subwords)
valid_indices = [i for i, tok in enumerate(tokens_cf) if not tok.startswith("##")]
if valid_indices:
    idx_max = valid_indices[np.argmax(abs_arr[valid_indices])]
else:
    idx_max = np.argmax(abs_arr)
token_to_change = tokens_cf[idx_max]
print(f"⚠ Top-impact token: '{token_to_change}'")

# 5.3: Enhanced antonym generation using contrastive prompting
mask_pipe = pipeline(
    "fill-mask",
    model="roberta-large",  # More capable model
    device=-1
)

def get_antonym_ml(word):
    """Improved antonym generation with correct mask token"""
    prompts = [
        f"Question: What is the direct antonym of '{word}'? Answer: <mask>",
        f"'{word}' → Opposite: <mask>",
        f"Antonym pair: {word} and <mask>",
        f"Positive: {word}, Negative: <mask>",
        f"If {word} means yes, then <mask> means no",
        f"Direct opposite of {word}: <mask>",
        f"Not {word} but <mask>"
    ]

    candidates = []
    for prompt in prompts:
        try:
            results = mask_pipe(prompt, top_k=5)
            for res in results:
                candidate = res["token_str"].strip().lower()
                if candidate != word.lower() and len(candidate) > 2:
                    candidates.append(candidate)
        except Exception as e:
            print(f"Warning: Error processing prompt '{prompt}': {str(e)}")

    if candidates:
        # Get most common candidate after basic filtering
        counter = Counter(candidates)
        for candidate, _ in counter.most_common():
            # Additional validation check
            val_prompt = f"Are '{word}' and '{candidate}' opposites? <mask>"
            val_results = mask_pipe(val_prompt, top_k=1)
            if "yes" not in val_results[0]["token_str"].lower():
                return candidate
        return counter.most_common(1)[0][0]

    return word  # Fallback if no candidates found

    # Find most frequent valid candidate
    if candidates:
        counter = Counter(candidates)
        for word, _ in counter.most_common():
            # Final validation using opposite context
            val_prompt = f"'{word}' is the opposite of '{word}'? [MASK]"
            val_results = mask_pipe(val_prompt)
            if "no" in val_results[0]["token_str"].lower():
                return word
        return counter.most_common(1)[0][0]
    return word  # Fallback

antonym = get_antonym_ml(token_to_change)
print(f"💡 Replacing '{token_to_change}' → '{antonym}'")

# 5.4: Case-sensitive replacement
cf_tokens = [
    antonym if tok == token_to_change else tok
    for tok in tokens_cf
]
cf_text = "".join(cf_tokens).replace("Ġ", " ").replace("▁", " ")  # Fix tokenization

# 5.5: Score and compare
orig_probs = predict_proba([text])[0]
cf_probs = predict_proba([cf_text])[0]

print("Original probs:      ", dict(zip(LABELS, orig_probs.round(3))))
print("Counterfactual probs:", dict(zip(LABELS, cf_probs.round(3))))

# 5.6: Visualization
plt.figure(figsize=(6,3))
x = np.arange(len(LABELS))
plt.bar(x - 0.2, orig_probs, 0.4, label='Original')
plt.bar(x + 0.2, cf_probs, 0.4, label='Counterfactual')
plt.xticks(x, LABELS)
plt.legend()
plt.title(f"Replacement: {token_to_change} → {antonym}")
plt.show()

In [ ]:
# ===== Modified Code: Remove Top SHAP Word =====


# 5.1: Extract SHAP explanation & tokens
sv_cf = explainer([text])[0]
shap_arr = sv_cf.values[:, pred_idx]
tokens_cf = list(sv_cf.data)
abs_arr = np.abs(shap_arr)

# 5.2: Find top-impact token (skip subwords)
valid_indices = [i for i, tok in enumerate(tokens_cf) if not tok.startswith("##")]
if valid_indices:
    idx_max = valid_indices[np.argmax(abs_arr[valid_indices])]
else:
    idx_max = np.argmax(abs_arr)
token_to_remove = tokens_cf[idx_max]
print(f"⚠ Top-impact token to remove: '{token_to_remove}'")

# 5.3: Create text with removed token
cf_tokens = [tok for tok in tokens_cf if tok != token_to_remove]

# Handle different tokenization schemes
if hasattr(sv_cf, 'data'):
    # For BERT-style tokenization
    cf_text = " ".join(cf_tokens).replace(" ##", "")
else:
    # For RoBERTa/GPT-style tokenization
    cf_text = "".join(cf_tokens).replace("Ġ", " ").replace("▁", " ").strip()

# 5.4: Score and compare
orig_probs = predict_proba([text])[0]
cf_probs = predict_proba([cf_text])[0]

print("Original text:", text)
print("Modified text:", cf_text)
print("\nOriginal probs:      ", dict(zip(LABELS, orig_probs.round(3))))
print("Modified probs:", dict(zip(LABELS, cf_probs.round(3))))

# 5.5: Visualization
plt.figure(figsize=(6,3))
x = np.arange(len(LABELS))
plt.bar(x - 0.2, orig_probs, 0.4, label='Original')
plt.bar(x + 0.2, cf_probs, 0.4, label='With Token Removed')
plt.xticks(x, LABELS)
plt.legend()
plt.title(f"Effect of removing '{token_to_remove}'")
plt.show()

## Section: Audio Counterfactual Analysis

In [ ]:
#  2) Load & normalize audio
audio_path = "03-01-05-01-01-02-01.wav"   # ← your file here
y, sr = librosa.load(audio_path, sr=16_000)
y = y / np.max(np.abs(y))                 # normalize to [–1,1]
print(f"Loaded {audio_path}: {len(y)} samples @ {sr} Hz")

In [ ]:
#  3) Feature-extract with frozen HuBERT
fe = Wav2Vec2FeatureExtractor.from_pretrained("facebook/hubert-large-ls960-ft")
hubert = HubertModel.from_pretrained("facebook/hubert-large-ls960-ft")
hubert.eval()
with torch.no_grad():
    inputs = fe(y, sampling_rate=sr, return_tensors="pt")
    input_values = inputs.input_values               # shape (1, seq_len)
    attention_mask = inputs.attention_mask
    X = hubert(input_values, attention_mask=attention_mask).last_hidden_state
    # X: (1, seq_len, hidden_size)

print("Extracted HuBERT features:", X.shape)

In [ ]:
#  4) Define BiLSTM head for 3-way sentiment
class AudioSentimentBiLSTM(nn.Module):
    def __init__(self, in_dim=1024, hidden_dim=256, num_classes=3, dropout=0.1):
        super().__init__()
        self.lstm    = nn.LSTM(input_size=in_dim, hidden_size=hidden_dim,
                               num_layers=1, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.fc      = nn.Linear(hidden_dim*2, num_classes)
    def forward(self, x):
        # x: (batch, seq_len, in_dim)
        lstm_out, _ = self.lstm(x)                 # (batch, seq_len, hidden*2)
        rep = lstm_out.mean(dim=1)                 # mean-pool → (batch, hidden*2)
        rep = self.dropout(rep)
        return self.fc(rep)                        # logits (batch, 3)

model = AudioSentimentBiLSTM(in_dim=X.shape[2])
model.eval()

In [ ]:
#  5) Inference
with torch.no_grad():
    logits = model(X)                               # (1,3)
    probs  = torch.softmax(logits, dim=-1).cpu().numpy().flatten()

labels = ["negative", "neutral", "positive"]
pred_idx = int(probs.argmax())
pred_cls = labels[pred_idx]
print("Sentiment probabilities:")
for lab,p in zip(labels, probs):
    print(f"  {lab:>8} → {p:.3f}")
print("Predicted class:", pred_cls)

In [ ]:
#  6) SHAP explainability over the feature‐sequence with GradientExplainer
# Ensure X requires gradients
X.requires_grad_()  # now X.grad will work

# Wrap model so SHAP can call it directly on tensor inputs
class LogitsWrapper(nn.Module):
    def __init__(self, submodel):
        super().__init__()
        self.sub = submodel
    def forward(self, x):
        return self.sub(x)

wrapped = LogitsWrapper(model)

# Use a zero‐background with the same shape as X
bg = torch.zeros_like(X)         # shape (1, seq_len, hidden)

# Switch to GradientExplainer to avoid the additivity assertion
ge = shap.GradientExplainer(wrapped, bg)

# Compute SHAP for full sequence
# returns a list of arrays, one per class, each of shape (1, seq_len, hidden)
shap_vals = ge.shap_values(X)

In [ ]:
#  7) Aggregate each time‐step by summing absolute contributions across hidden dim
sv = np.abs(shap_vals[pred_idx]).sum(axis=-1).flatten()  # → (seq_len,)

In [ ]:
#  8) Build time axes
audio_dur = len(y) / sr
t_feat    = np.linspace(0, audio_dur, sv.shape[0])
t_audio   = np.linspace(0, audio_dur, len(y))

In [ ]:
#  9) Simple counterfactual: scale amplitude by 0.5
cf_y = y * 0.5

In [ ]:
#  10) Plot SHAP contributions and waveforms

plt.figure(figsize=(12,6))

# SHAP contributions
plt.subplot(2,1,1)
plt.plot(t_feat, sv, linewidth=2)
plt.xlabel("Time (s)")
plt.ylabel("Aggregate |SHAP|")
plt.title(f"SHAP contributions → '{pred_cls}'")

# Original vs CF waveform
plt.subplot(2,1,2)
plt.plot(t_audio, y,    label="Original")
plt.plot(t_audio, cf_y, label="Counterfactual (×0.5)")
plt.xlabel("Time (s)")
plt.legend()
plt.title("Waveforms")

plt.tight_layout()
plt.show()

## Section: Video Counterfactual Analysis

In [ ]:
!apt-get update                                  # FFmpeg libs for PyAV
!apt-get install -y libavdevice-dev libavfilter-dev libavformat-dev libavcodec-dev

In [ ]:
# ─── 1) TAKE VIDEO AS INPUT ────────────────────────────────────────────────────
VIDEO_IN = "videoplayback.mp4"    # put your filename here

In [ ]:
# ─── 2) EXTRACT FRAMES & SAMPLE TO 100 ───────────────────────────────────────
def extract_sampled_frames(path, size=(224, 224), num_frames=100):
    cap = cv2.VideoCapture(path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    idxs = np.linspace(0, total-1, num=num_frames, dtype=int)
    frames = []
    for i in range(total):
        ret, frame = cap.read()
        if not ret:
            break
        if i in idxs:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, size)
            frames.append(frame)
    cap.release()
    return frames

frames = extract_sampled_frames(VIDEO_IN, num_frames=100)

In [ ]:
# Load CLIP model + processor (zero‐shot)
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_proc  = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Sentiment labels for zero‐shot
sentiment_labels = ["positive", "neutral", "negative"]
text_inputs = clip_proc(text=sentiment_labels, return_tensors="pt", padding=True).to(device)

In [ ]:
# ─── 3) MODEL: ZERO‐SHOT SENTIMENT PER FRAME ──────────────────────────────────
def predict_frame_sentiment(frame):
    # returns raw scores for each label
    inputs = clip_proc(images=frame, return_tensors="pt").to(device)
    with torch.no_grad():
        img_emb = clip_model.get_image_features(**inputs)            # (1, dim)
        txt_emb = clip_model.get_text_features(**text_inputs)       # (3, dim)
    # cosine similarity
    scores = (img_emb @ txt_emb.T)[0] / (img_emb.norm() * txt_emb.norm(dim=1))
    probs  = torch.softmax(scores, dim=0).cpu().numpy()
    return dict(zip(sentiment_labels, probs))

# Compute original video probabilities by averaging frames
orig_frame_probs = np.array([list(predict_frame_sentiment(f).values()) for f in frames])
orig_probs = orig_frame_probs.mean(axis=0)
orig_dict  = dict(zip(sentiment_labels, orig_probs))
print("Original video sentiment:", orig_dict)

In [ ]:
# ─── 4) SHAPLEY VALUES FRAME-BY-FRAME ─────────────────────────────────────────
# Use mean-pixel intensities as proxy features for each frame
frame_feats = np.stack([f.mean(axis=(0,1)).flatten() for f in frames])

# SHAP explainer: model_predict maps feats→score(“positive”) for simplicity
def model_predict(feat_array):
    # reconstruct gray frames from feature
    gray_frames = [(np.ones((100,100,3))*feat.mean()).astype(np.uint8) for feat in feat_array]
    # get probability of 'positive' label
    return np.array([predict_frame_sentiment(f)["positive"] for f in gray_frames])

explainer = shap.KernelExplainer(model_predict, frame_feats)
shap_vals = explainer.shap_values(frame_feats)  # array of shape (n_frames, feat_dim)

In [ ]:
# ─── 5) PLOT SHAPLEY VALUE GRAPH ──────────────────────────────────────────────
mean_abs_shap = np.mean(np.abs(shap_vals), axis=1)  # one value per frame
plt.figure(figsize=(10,4))
plt.plot(mean_abs_shap, label="|SHAP| per frame")
plt.xlabel("Frame index")
plt.ylabel("Mean |SHAP value|")
plt.title("Frame-by-Frame Feature Importance")
plt.legend()
plt.show()

In [ ]:
# ─── 6) COUNTERFACTUAL: FRAMES ALREADY EXTRACTED ───────────────────────────────
# ─── 7) NOISE: GAUSSIAN & SALT-AND-PEPPER ──────────────────────────────────────
def add_noise(frames, noise="gaussian", var=0.05):
    out = []
    for f in frames:
        f_norm = f.astype(np.float32) / 255.0
        if noise=="gaussian":
            g = np.random.normal(0, var**0.5, f_norm.shape)
            nf = np.clip(f_norm + g, 0,1)
        else:
            nf = f_norm.copy()
            rnd = np.random.rand(*f_norm.shape[:2])
            nf[rnd<var/2]       = 0
            nf[rnd>1-var/2]     = 1
        out.append((nf*255).astype(np.uint8))
    return out

cf_frames = add_noise(frames, noise="gaussian", var=0.05)

In [ ]:
# ─── 7b) WRITE COUNTERFACTUAL VIDEO ────────────────────────────────────────────
def write_video(frames, out_file, fps=25):
    h,w = frames[0].shape[:2]
    writer = cv2.VideoWriter(out_file, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w,h))
    for f in frames:
        writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
    writer.release()

CF_OUT = "cf_video.mp4"
write_video(cf_frames, CF_OUT)

In [ ]:
# ─── 8) COMPARE ORIGINAL vs COUNTERFACTUAL SENTIMENT ──────────────────────────
cf_frame_probs = np.array([list(predict_frame_sentiment(f).values()) for f in cf_frames])
cf_probs       = cf_frame_probs.mean(axis=0)
cf_dict        = dict(zip(sentiment_labels, cf_probs))
print("Counterfactual video sentiment:", cf_dict)

# Plot comparison
x = np.arange(len(sentiment_labels))
width = 0.35
plt.figure(figsize=(6,4))
plt.bar(x-width/2, orig_probs, width, label="Original")
plt.bar(x+width/2, cf_probs,   width, label="Counterfactual")
plt.xticks(x, sentiment_labels)
plt.ylabel("Probability")
plt.title("Original vs Counterfactual Sentiment")
plt.legend()
plt.show()

# Zoom

In [ ]:
# Load OpenCV’s face detector
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

def get_counterfactual_frames(frames, mode="tight", margin=0.2):
    """
    mode="tight": crop exactly to detected face
    mode="zoom_out": expand crop by `margin` fraction to include more background
    """
    cf = []
    for f in frames:
        gray = cv2.cvtColor(f, cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
        if len(faces) > 0:
            x, y, w, h = faces[0]
            if mode == "tight":
                crop = f[y : y + h, x : x + w]
            else:  # zoom_out
                mx = int(w * margin)
                my = int(h * margin)
                x1 = max(0, x - mx)
                y1 = max(0, y - my)
                x2 = min(f.shape[1], x + w + mx)
                y2 = min(f.shape[0], y + h + my)
                crop = f[y1:y2, x1:x2]
        else:
            # no face detected, fallback to full frame
            crop = f
        # resize back to model input size
        cf.append(cv2.resize(crop, (224, 224)))
    return cf

# Generate your counterfactual frames:
# For a tight face crop:
cf_frames = get_counterfactual_frames(frames, mode="tight")

# Or, to zoom out and include more background:
# cf_frames = get_counterfactual_frames(frames, mode="zoom_out", margin=0.3)

In [ ]:
# ── then write and predict exactly as before ─────────────────────────────────

CF_OUT = "cff_video.mp4"
write_video(cf_frames, CF_OUT)

cf_frame_probs = np.array([list(predict_frame_sentiment(f).values()) for f in cf_frames])
cf_probs = cf_frame_probs.mean(axis=0)
cf_dict = dict(zip(sentiment_labels, cf_probs))

print("Counterfactual (face-cropped) sentiment:", cf_dict)

In [ ]:
# Plot comparison
x = np.arange(len(sentiment_labels))
width = 0.35
plt.figure(figsize=(6, 4))
plt.bar(x - width / 2, orig_probs, width, label="Original")
plt.bar(x + width / 2, cf_probs, width, label="Counterfactual")
plt.xticks(x, sentiment_labels)
plt.ylabel("Probability")
plt.title("Original vs Counterfactual Sentiment")
plt.legend()
plt.show()

# Color Change

In [ ]:
def change_clothing_color(frames, hue_shift=60):
    """
    Shifts the hue of the entire frame by `hue_shift` degrees in HSV space,
    which effectively remaps clothing (and everything else) to a new color.
    You can tweak hue_shift (0-180) to target different color changes.
    """
    cf = []
    for f in frames:
        # Convert to HSV
        hsv = cv2.cvtColor(f, cv2.COLOR_RGB2HSV).astype(np.uint16)
        h, s, v = cv2.split(hsv)

        # Shift hue channel
        h = (h + hue_shift) % 180

        # Merge and convert back to RGB
        hsv_shifted = cv2.merge([h.astype(np.uint8), s.astype(np.uint8), v.astype(np.uint8)])
        rgb_shifted = cv2.cvtColor(hsv_shifted, cv2.COLOR_HSV2RGB)
        cf.append(rgb_shifted)
    return cf

# Generate your counterfactual frames by shifting hue by 60° (e.g., blue→green)
cf_frames = change_clothing_color(frames, hue_shift=60)

In [ ]:
# ── then write and predict exactly as before ───────────────────────────────────

CF_OUT = "cffc_video.mp4"
write_video(cf_frames, CF_OUT)

cf_frame_probs = np.array([list(predict_frame_sentiment(f).values()) for f in cf_frames])
cf_probs       = cf_frame_probs.mean(axis=0)
cf_dict        = dict(zip(sentiment_labels, cf_probs))

print("Counterfactual (clothing-color-changed) sentiment:", cf_dict)

In [ ]:
# Comparison plot (same as before)
x = np.arange(len(sentiment_labels))
width = 0.35
plt.figure(figsize=(6,4))
plt.bar(x-width/2, orig_probs, width, label="Original")
plt.bar(x+width/2, cf_probs,   width, label="Counterfactual")
plt.xticks(x, sentiment_labels)
plt.ylabel("Probability")
plt.title("Original vs Counterfactual Sentiment")
plt.legend()
plt.show()